# Index Types

* FLAT

In [55]:
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection, utility
import random

In [56]:
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

In [57]:
utility.list_collections()

['Spam_Test', 'Album1', 'dynamic_schema_example', 'partition_key_collection']

### Make a copy of Spam_Test

In [58]:
utility.has_collection("Spam_Test")

True

In [59]:
collection_spam = Collection("Spam_Test")
schema_spam = collection_spam.schema

In [60]:
indexes = collection_spam.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")

Index Name: IVF_FLAT
Index Params: {'metric_type': 'L2', 'index_type': 'IVF_FLAT', 'params': {'nlist': 1024}, 'index_name': 'SMS_IVF_FLAT_TEST'}



In [61]:
for field in collection_spam.schema.fields:
    print(f"Field Name: {field.name}")
    print(f"Field Type: {field.dtype}")
    print(f"Field Description: {field.description}")
    print(f"Is Primary: {field.is_primary}")
    print(f"Auto ID: {field.auto_id}")
    print("")

Field Name: id
Field Type: 5
Field Description: 
Is Primary: True
Auto ID: False

Field Name: message
Field Type: 21
Field Description: 
Is Primary: False
Auto ID: False

Field Name: message_embeddings
Field Type: 101
Field Description: 
Is Primary: False
Auto ID: False



In [62]:
collection_spam.load()

In [63]:
# Retrieve data from the original collection
query_expr = "id >= 1"
num_entities = collection_spam.num_entities
data = collection_spam.query(expr=query_expr, output_fields=["id", "message", "message_embeddings"], limit=num_entities)

In [64]:
# Extract fields from the query result
ids = [item['id'] for item in data]
messages = [item['message'] for item in data]
message_embeddings = [item['message_embeddings'] for item in data]

In [66]:
# Create the new collection with the same schema
collection_spam_copy = Collection(name="Spam_Test_Copy", schema=schema_spam)

In [67]:
# Insert the data into the new collection
insert_data = [ids, messages, message_embeddings]
collection_spam_copy.insert(insert_data)

(insert count: 5574, delete count: 0, upsert count: 0, timestamp: 450979404863766532, success count: 5574, err count: 0)

In [68]:
collection_spam.release()

In [69]:
indexes = collection_spam_copy.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")
len(indexes)

0

### Get test message vector

In [70]:
# Read the file and load it into a list - 'claim prize' vector
with open("test_message_vector.txt", "r") as file:
    test_message_vector = file.read().splitlines()

test_message_vector = [float(i) for i in test_message_vector]

## Flat Index

It is the simplest of all index types in Milvus.

In fact, flat index does nothing on the vectors that are indexed and hence named as flat.

With FLAT index, we will calculate the distance between the query vector and all the vectors that are part of the dataset. Based on the distance with query vector, we will find the top three nearest neighbors to the query vector.

For vector similarity search applications that require perfect accuracy and depend on relatively small (million-scale) datasets, the FLAT index is a good choice. FLAT does not compress vectors, and is the only index that can guarantee exact search results. Results from FLAT can also be used as a point of comparison for results produced by other indexes that have less than 100% recall.

FLAT is accurate because it takes an exhaustive approach to search, which means for each query the target input is compared to every set of vectors in a dataset. This makes FLAT the slowest index on our list, and poorly suited for querying massive vector data. There are no parameters required for the FLAT index in Milvus, and using it does not need data training.

In [71]:
# FLAT 

# Create Index
index_params_flat = {
    "metric_type": "L2",
    "index_type": "FLAT",
    "params": {},
    "index_name": "FLAT_EXAMPLE_INDEX"
}

# Index on vector field
collection_spam_copy.create_index(
    field_name="message_embeddings",
    index_params=index_params_flat
)

# Vector similarity search
search_params = {"metric_type": "L2", "params": {}}

collection_spam_copy.load()

results = collection_spam_copy.search(
    data=[test_message_vector],
    anns_field="message_embeddings",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=['message']
)

In [72]:
for result in results[0]:
    print (result)

id: 1119, distance: 1.1402018070220947, entity: {'message': '449050000301 You have won a Â£2,000 price! To claim, call 09050000301.'}
id: 577, distance: 1.1955623626708984, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
id: 4048, distance: 1.2085381746292114, entity: {'message': 'Win a Â£1000 cash prize or a prize worth Â£5000'}
id: 651, distance: 1.23152494430542, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327. T&C: RSTM, SW7 3SS. 150ppm'}
id: 9, distance: 1.2500566244125366, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}


In [73]:
indexes = collection_spam_copy.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")

Index Name: FLAT
Index Params: {'metric_type': 'L2', 'index_type': 'FLAT', 'params': {}, 'index_name': 'FLAT_EXAMPLE_INDEX'}



In [74]:
# Release the entire collection data
collection_spam_copy.release()

# Drop the index
collection_spam_copy.drop_index()

# Check the index
collection_spam_copy.has_index() # False

False

## IVF_FLAT

IVF_FLAT divides vector data into nlist cluster units, and then compares distances between the target input vector and the center of each cluster. Depending on the number of clusters the system is set to query (nprobe), similarity search results are returned based on comparisons between the target input and the vectors in the most similar cluster(s) only — drastically reducing query time.

By adjusting nprobe, an ideal balance between accuracy and speed can be found for a given scenario. Results from the IVF_FLAT performance test demonstrate that query time increases sharply as both the number of target input vectors (nq), and the number of clusters to search (nprobe), increase.

IVF_FLAT is the most basic IVF index, and the encoded data stored in each unit is consistent with the original data.

In [75]:
# IVF_FLAT 

# Create Index
index_params_ivf_flat = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist":1024},  # nlist: Number of cluster units
    "index_name": "IVF_FLAT_EXAMPLE_INDEX"
}

# Index on vector field
collection_spam_copy.create_index(
    field_name="message_embeddings",
    index_params=index_params_ivf_flat
)

# Vector similarity search
search_params = {"metric_type": "L2", "params": {"nprobe": 64}}  # nprobe: Number of units to query

collection_spam_copy.load()

results = collection_spam_copy.search(
    data=[test_message_vector],
    anns_field="message_embeddings",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=['message']
)

In [76]:
for result in results[0]:
    print (result)

id: 1119, distance: 1.1402018070220947, entity: {'message': '449050000301 You have won a Â£2,000 price! To claim, call 09050000301.'}
id: 577, distance: 1.1955623626708984, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
id: 4048, distance: 1.2085381746292114, entity: {'message': 'Win a Â£1000 cash prize or a prize worth Â£5000'}
id: 651, distance: 1.23152494430542, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327. T&C: RSTM, SW7 3SS. 150ppm'}
id: 9, distance: 1.2500566244125366, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}


In [77]:
indexes = collection_spam_copy.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")

Index Name: IVF_FLAT
Index Params: {'metric_type': 'L2', 'index_type': 'IVF_FLAT', 'params': {'nlist': 1024}, 'index_name': 'IVF_FLAT_EXAMPLE_INDEX'}



In [78]:
# Release the entire collection data
collection_spam_copy.release()

# Drop the index
collection_spam_copy.drop_index()

# Check the index
collection_spam_copy.has_index() # False

False

## IVF_SQ8

IVF_FLAT does not perform any compression, so the index files it produces are roughly the same size as the original, raw non-indexed vector data. For example, if the original 1B SIFT dataset is 476 GB, its IVF_FLAT index files will be slightly smaller (~470 GB). Loading all the index files into memory will consume 470 GB of storage.

When disk, CPU, or GPU memory resources are limited, IVF_SQ8 is a better option than IVF_FLAT. This index type can convert each FLOAT (4 bytes) to UINT8 (1 byte) by performing Scalar Quantization (SQ). This reduces disk, CPU, and GPU memory consumption by 70–75%. For the 1B SIFT dataset, the IVF_SQ8 index files require just 140 GB of storage.

In [79]:
# IVF_SQ8 

# Create Index
index_params_ivf_sq8 = {
    "metric_type": "L2",
    "index_type": "IVF_SQ8",
    "params": {"nlist":1024},  # nlist: Number of cluster units
    "index_name": "IVF_SQ8_EXAMPLE_INDEX"
}

# Index on vector field
collection_spam_copy.create_index(
    field_name="message_embeddings",
    index_params=index_params_ivf_sq8
)

# Vector similarity search
search_params = {"metric_type": "L2", "params": {"nprobe": 64}}  # nprobe: Number of units to query

collection_spam_copy.load()

results = collection_spam_copy.search(
    data=[test_message_vector],
    anns_field="message_embeddings",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=['message']
)

In [80]:
for result in results[0]:
    print (result)

id: 1119, distance: 1.1402018070220947, entity: {'message': '449050000301 You have won a Â£2,000 price! To claim, call 09050000301.'}
id: 577, distance: 1.1955623626708984, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
id: 4048, distance: 1.2085381746292114, entity: {'message': 'Win a Â£1000 cash prize or a prize worth Â£5000'}
id: 651, distance: 1.23152494430542, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327. T&C: RSTM, SW7 3SS. 150ppm'}
id: 9, distance: 1.2500566244125366, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}


In [81]:
indexes = collection_spam_copy.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")

Index Name: IVF_SQ8
Index Params: {'metric_type': 'L2', 'index_type': 'IVF_SQ8', 'params': {'nlist': 1024}, 'index_name': 'IVF_SQ8_EXAMPLE_INDEX'}



In [83]:
# Release the entire collection data
collection_spam_copy.release()

# Drop the index
collection_spam_copy.drop_index()

# Check the index
collection_spam_copy.has_index() # False

False

## IVF_PQ

PQ (Product Quantization) uniformly decomposes the original high-dimensional vector space into Cartesian products of m low-dimensional vector spaces, and then quantizes the decomposed low-dimensional vector spaces. Instead of calculating the distances between the target vector and the center of all the units, product quantization enables the calculation of distances between the target vector and the clustering center of each low-dimensional space and greatly reduces the time complexity and space complexity of the algorithm.

IVF_PQ performs IVF index clustering before quantizing the product of vectors. Its index file is even smaller than IVF_SQ8, but it also causes a loss of accuracy during searching vectors.

In [84]:
# IVF_PQ

# Create Index
index_params_ivf_pq = {
    "metric_type": "L2",
    "index_type": "IVF_PQ",
    "params": {"nlist":1024, "m":2, "nbits": 16},  # nlist: Number of cluster units, m:Number of factors of product quantization, nbits: Number of bits in which each low-dimensional vector is stored.
    "index_name": "IVF_PQ_EXAMPLE_INDEX"
}

# Index on vector field
collection_spam_copy.create_index(
    field_name="message_embeddings",
    index_params=index_params_ivf_pq
)

# Vector similarity search
search_params = {"metric_type": "L2", "params": {"nprobe": 64}}  # nprobe: Number of units to query

collection_spam_copy.load()

results = collection_spam_copy.search(
    data=[test_message_vector],
    anns_field="message_embeddings",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=['message']
)

In [85]:
for result in results[0]:
    print (result)

id: 1119, distance: 1.1402018070220947, entity: {'message': '449050000301 You have won a Â£2,000 price! To claim, call 09050000301.'}
id: 577, distance: 1.1955623626708984, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
id: 4048, distance: 1.2085381746292114, entity: {'message': 'Win a Â£1000 cash prize or a prize worth Â£5000'}
id: 651, distance: 1.23152494430542, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327. T&C: RSTM, SW7 3SS. 150ppm'}
id: 9, distance: 1.2500566244125366, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}


In [86]:
indexes = collection_spam_copy.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")

Index Name: IVF_PQ
Index Params: {'metric_type': 'L2', 'index_type': 'IVF_PQ', 'params': {'nlist': 1024, 'm': 2, 'nbits': 16}, 'index_name': 'IVF_PQ_EXAMPLE_INDEX'}



In [87]:
# Release the entire collection data
collection_spam_copy.release()

# Drop the index
collection_spam_copy.drop_index()

# Check the index
collection_spam_copy.has_index() # False

False

## HNSW (Hierarchical Navigable Small World Graph)

HNSW (Hierarchical Navigable Small World Graph) is a graph-based indexing algorithm. It builds a multi-layer navigation structure for an image according to certain rules. In this structure, the upper layers are more sparse and the distances between nodes are farther; the lower layers are denser and the distances between nodes are closer. The search starts from the uppermost layer, finds the node closest to the target in this layer, and then enters the next layer to begin another search. After multiple iterations, it can quickly approach the target position.

In order to improve performance, HNSW limits the maximum degree of nodes on each layer of the graph to M. In addition, you can use efConstruction (when building index) or ef (when searching targets) to specify a search range.

In [88]:
# HNSW

# Create Index
index_params_hnsw = {
    "metric_type": "L2",
    "index_type": "HNSW",
    "params": {"M":32, "efConstruction":16},  # M: tha maximum number of outgoing connections in the graph. ef_construction: controls index search speed/build speed tradeoff. Increasing the efConstruction parameter may enhance index quality, but it also tends to lengthen the indexing time.
    "index_name": "IVF_PQ_EXAMPLE_INDEX"
}

# Index on vector field
collection_spam_copy.create_index(
    field_name="message_embeddings",
    index_params=index_params_hnsw
)

# Vector similarity search
search_params = {"metric_type": "L2", "params": {"ef": 4}}  # ef: Parameter controlling query time/accuracy trade-off. Higher ef leads to more accurate but slower search.

collection_spam_copy.load()

results = collection_spam_copy.search(
    data=[test_message_vector],
    anns_field="message_embeddings",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=['message']
)

In [89]:
for result in results[0]:
    print (result)

id: 1119, distance: 1.1402018070220947, entity: {'message': '449050000301 You have won a Â£2,000 price! To claim, call 09050000301.'}
id: 577, distance: 1.1955623626708984, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327'}
id: 4048, distance: 1.2085381746292114, entity: {'message': 'Win a Â£1000 cash prize or a prize worth Â£5000'}
id: 651, distance: 1.23152494430542, entity: {'message': 'You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327. T&C: RSTM, SW7 3SS. 150ppm'}
id: 9, distance: 1.2500566244125366, entity: {'message': 'WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'}


In [90]:
indexes = collection_spam_copy.indexes
for index in indexes:
    print(f"Index Name: {index.params['index_type']}")
    print(f"Index Params: {index.params}")
    print("")

Index Name: HNSW
Index Params: {'metric_type': 'L2', 'index_type': 'HNSW', 'params': {'M': 32, 'efConstruction': 16}, 'index_name': 'IVF_PQ_EXAMPLE_INDEX'}



In [91]:
# Release the entire collection data
collection_spam_copy.release()

# Drop the index
collection_spam_copy.drop_index()

# Check the index
collection_spam_copy.has_index() # False

False